## 6. Classification
### 1. Merge one hot encoded genres to lyrics

In [1]:
import pandas as pd

lyrics = pd.read_csv('../dataset/Lyrics_en_clean.csv', index_col=['Band','Song'])
genres = pd.read_csv('../dataset/artists_final_ohe.csv', index_col=['Band'])

#df = pd.concat([lyrics, genres], axis=1, join_axes=[lyrics.index], levels=lyrics.index.levels)
df = pd.merge(lyrics, genres, on=['Band'])
del lyrics
del genres
df

,Lyrics,'n,a,abstract,accordion,acid,acoustic,adoracao,adult,african,...,western,wonky,world,worship,wrestling,ye,zapstep,zolo,zouk,zydeco
Band,,,,,,,,,,,,,,,,,,,,,
Elijah Blake,"No, no I aint ever trapped out the bando But o...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,"The drinks go down and smoke goes up, I feel m...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,She dont live on planet Earth no more She foun...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,"Trippin off that Grigio, mobbin, lights low Tr...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,"I see a midnight panther, so gallant and so br...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,I just want to ready your mind Cause Ill still...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,"Keep rockin we gone, keep rockin we gone Keep ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,Shes got no soul but still young minded Yeah s...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elijah Blake,Im so sick without ya baby Not even a doctor c...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Label exploration  

In [2]:
%matplotlib inline
data = df.drop(['Lyrics'], axis = 1)
#pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [5]:
import seaborn as sns
sns.heatmap(data.corr())

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from sklearn.decomposition import PCA
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np


good_data = data
pca = PCA()
pca.fit(good_data)

# Dimension indexing
dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

# PCA components
components = pd.DataFrame(np.round(pca.components_, 4), columns = list(good_data.keys()))
components.index = dimensions

# PCA explained variance
ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
variance_ratios.index = dimensions

# Create a bar plot visualization
fig, ax = plt.subplots(figsize = (14,8))

# Plot the feature weights as a function of the components
components.plot(ax = ax, kind = 'bar');
ax.set_ylabel("Feature Weights")
ax.set_xticklabels(dimensions, rotation=0)


# Display the explained variance ratios
for i, ev in enumerate(pca.explained_variance_ratio_):
    ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

# Return a concatenated DataFrame
pd.concat([variance_ratios, components], axis = 1)

In [ ]:
from sklearn.cluster import FeatureAgglomeration

clf = FeatureAgglomeration(n_components=200)
reduced_data = clf.fit_transform(data)  

### 3. Doc2Vec Deep learning LSTM classifier

In [3]:
y_train = df.drop(['Lyrics'], axis=1)
del df

In [4]:
from gensim.models.doc2vec import Doc2Vec

doc2vec = Doc2Vec.load("../dataset/doc2vec")
doc_vectors = doc2vec.wv
print("Number of doc vectors: {}".format(len(doc_vectors.vocab)))

C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Number of doc vectors: 67565


In [5]:
import numpy as np
Text_INPUT_DIM = 100
train_size = y_train.shape[0] 
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = doc2vec.docvecs[i]

In [6]:
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model_conv = Sequential()
model_conv.add(Embedding(input_dim=train_size, output_dim=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(y_train.shape[1], activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()

estimator = model_conv.fit(text_train_arrays, y_train, validation_split=0.2, epochs = 2, batch_size=1024)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         33366600  
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 646)               65246     
Total params: 33,529,910
Trainable params: 33,529,910
Non-trainable params: 0
________________________________________________________________

In [8]:
#Bryce Fox
#Lucy
#genres: "indie poptimism","modern alternative rock","modern rock"
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm

lyric = "So bring your vibe over here here here \
Throw it up in the air air air, oh \
I run my hands through your hair hair hair \
And give you that love cuz you're oh so rare \
You look better over here then take you do over there \
And I don't wanna share \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I'll fix you up something real real tall \
It's looking up, we should take it all off \
Yeah \
You remind me why the stars don't fall \
I swear I'll love you like your last name's Ball \
I'll take you to the lair if we make it past the stairs \
I don't wanna share, yeah \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
 \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
Yeah \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
(So bite your tongue before I bite it for ya) \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
She said she ain't Lucy \
I said not yet" 

lyric = simple_preprocess(str(lyric), deacc=True)
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['s', 'chorus'])

new_lyric = list()
for word in lyric:
    if(word not in stop_words):
        new_lyric.append(word)
lyric = new_lyric


# python -m spacy download en
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']

lemma_lyric = list()
doc = nlp(" ".join(lyric)) 
lemma_lyric = list()
for token in doc:
    if(token.pos_ in allowed_postags):
        lemma_lyric.append(token.lemma_)
lyric = lemma_lyric
print(lyric)

C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['bring', 'vibe', 'throw', 'air', 'air', 'air', 'run', 'hand', 'hair', 'hair', 'hair', 'give', 'love', 'rare', 'look', 'better', 'take', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'fix', 'something', 'real', 'real', 'tall', 'look', 'take', 'remind', 'star', 'fall', 'swear', 'love', 'last', 'name', 'ball', 'take', 'lair', 'make', 'past', 'stair', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil

In [9]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.4380852,
 'pop': 0.35069844,
 'alternative': 0.15339598,
 'dance': 0.124250904,
 'new': 0.12347917,
 'folk': 0.120172046,
 'jazz': 0.11210153,
 'christmas': 0.11025446,
 'mellow': 0.10850386,
 'metal': 0.10696753}